![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
ranks = pd.read_csv("/kaggle/input/ta-ranks/ranks.csv").drop(['Unnamed: 0','claimed'], axis=1).drop_duplicates(subset=['url'])
wordcounts = pd.read_csv("/kaggle/input/ta-keywords/words_keys.csv", sep=';').drop(['Unnamed: 0'],axis=1)
cities = pd.read_csv('/kaggle/input/world-cities-datasets/worldcities.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')


df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
#Additional data sets
ranks = pd.read_csv("/kaggle/input/ta-ranks/ranks.csv").drop(['Unnamed: 0','claimed'], axis=1).drop_duplicates(subset=['url'])
wordcounts = pd.read_csv("/kaggle/input/ta-keywords/words_keys.csv", sep=';').drop(['Unnamed: 0'],axis=1)
cities = pd.read_csv('/kaggle/input/world-cities-datasets/worldcities.csv')

In [ ]:
# Конвертация уровня стоимости
def conv_price(x):
    d = {
        '$$ - $$$':2,
        '$': 1,
        '$$$$':3
    }
    if x in d.keys():
        return d[x]
    elif pd.isnull(x):
        return float('NaN')
    else:
        return float('NaN')
# Конвертация Cuisine Style
def conv_cuisine(x):
    lx=re.compile('\[.*\]')
    if type(x)==str and lx.fullmatch(x):
        lst=eval(x)
        return lst
    #elif pd.isnull(x):
    else:
        return ['NaN']

In [ ]:
def gen_values_set(data, colname):
    vals=set()
    lx=re.compile('\[.*\]')
    for v in data[colname]:
        if type(v)==str and lx.fullmatch(v):
            lst=eval(v)
            for x in lst:
                vals.add(x)
        elif type(v)==list:
            lst=v
            for x in lst:
                vals.add(x)
        else:
            vals.add(v)
    return vals
# Функция создания Dummies (поддерживает списки)
def gen_valcols(colname, df):
    vals=gen_values_set(df, colname)

    def find_item(item):
        if type(item)==str:
            if v==item:
                return 1
            else:
                return 0
        elif type(item)==list:
            if v in item:
                return 1
            else:
                return 0
            
    for v in vals:
        df[colname+v]=df[colname].apply(lambda x: find_item(x))

In [ ]:
lrx=re.compile('\[\[.*\]\]')
def extr_rev(row):
    #print(row)
    x=row['Reviews']
    lst=[[],[]]
    if type(x)== str and lrx.fullmatch(x):
        nan=''
        lst=eval(x)
#CORR
#    row['rev_count'] =  len(lst[0])
    row['rev1'] = lst[0][0] if len(lst[0])>0 else ''
    row['rev2'] = lst[0][1] if len(lst[0])>1 else ''
    
    row['date1'] = pd.to_datetime(lst[1][0] if len(lst[1])>0 else '', format='%m/%d/%Y', errors='coerce')
    row['date2'] = pd.to_datetime(lst[1][1] if len(lst[1])>1 else '')
    
    row['date1']= pd.to_datetime(row['date1'])
    row['date2']=pd.to_datetime(row['date2'])
    
    return row

In [ ]:

def get_date_delta(row, date_min, date_max):
    date1=row['date1']
    date2=row['date2']
    delta = date_max-date_min
    if not pd.isnull(date1) and not pd.isnull(date2):
        #print(date2, date1, date2-date1)
        delta = date1 - date2 if date2<date1 else date2-date1
    elif not pd.isnull(date1):
        delta = date1 - date_min
    return delta.total_seconds()

def get_last_date_delta(row, date_min, date_max):
    date1=row['date1']
    date2=row['date2']
    last_date=date_min
    if not pd.isnull(date1):
        last_date=date1
    if (not pd.isnull(date2)) and date2 > date1:
        last_date=date2
#    return (datetime.now()-last_date).total_seconds()
    return (date_max-last_date).total_seconds()

def get_first_date_delta(row, date_min, date_max):
    date1=row['date1']
    date2=row['date2']
    first_date=date_min
    if not pd.isnull(date1):
        first_date=date1
    if (not pd.isnull(date2)) and date2 < date1:
        first_date=date2
    return (first_date - date_min).total_seconds()

In [ ]:
allwords=pd.Series()
def collectwords(x):
    print(type(x))
    global allwords
    #x=map(lambda s: s.lower(), x)
    #s=pd.Series(x)
    allwords=allwords.append(x.str.lower())
    
def get_dummies_words(df,colname, words, init=True):
    words_re=re.compile('\w+')
    def find_item(row):
        item=row[colname]
        if type(item)==str:
            lst=list(words_re.findall(item.lower()))
            count = 0
#            for v in words:
#                if v in lst:
#                #if v in item.lower():
#                    
#                    row[prefix+v]=row[prefix+v]+1
#                    count = count + 1
            for v in lst:
                if v in words:
                    row[prefix+v]=row[prefix+v]+1
                    count = count + 1
            row[prefix + 'COUNT']=row[prefix + 'COUNT']+len(lst)
            row[prefix + 'NOWORDS']= row[prefix + 'NOWORDS'] + (1 if count == 0 else 0)
        return row
        
    prefix='word_'
    if init:
        for v in words:
            df[prefix+v]=0
            df[prefix + 'COUNT']=0
            df[prefix + 'NOWORDS']=0
            
    df=df.apply(find_item, axis=1)
    return df

In [ ]:
def is_closed(x):
    if pd.isna(x['rank1']) and pd.isna(x['rank2']) and pd.isna(x['rank_main']):
        return 1
    else:
        return 0

def top_rank(x):
    top_rank = x['Ranking']
    if not(pd.isna(x['rank1'])) and top_rank < x['rank1']:
        top_rank = x['rank1']
    if not(pd.isna(x['rank2'])) and top_rank < x['rank2']:
        top_rank = x['rank2']
    if not(pd.isna(x['rank_main'])) and top_rank < x['rank_main']:
        top_rank = x['rank_main']
    return float(top_rank)

def get_rank_norm(x, field, ranks_by_city):
    maxr = ranks_by_city.loc[x['City']][( field, 'max')] / ranks_by_city.loc[x['City']][( 'population', 'max')]
    minr = ranks_by_city.loc[x['City']][( field, 'min')] / ranks_by_city.loc[x['City']][( 'population', 'max')]
    res=x[field]/ ranks_by_city.loc[x['City']][( 'population', 'max')]
    return (res - minr) / (maxr - minr)

In [ ]:
def round_rating(x):
    return (round(x*2.0)/2)

def norm(x, fieldname):
    return (x[fieldname]-x[fieldname].min())/(x[fieldname].max()-x[fieldname].min())

In [ ]:
def is_local_cuisine(row):
    local_cuisine_by_country = {
        'United Kingdom':['British','Scottish'],
        'Spain': ['Spanish'],
        'France': ['French','Central European'], 
        'Italy': ['Italian','Central European'],
        'Germany': ['Dutch','German','Central European'],
        'Portugal': ['Portuguese'],
        'Czechia': ['Czech','Eastern European'],
        'Poland':['Polish','Eastern European'],
        'Austria': ['Austrian','Central European'],
        'Netherlands':['Scandinavian'],
        'Belgium': ['Belgian','Eastern European'],
        'Switzerland':['Swiss','Central European'],
        'Sweden':['Scandinavian'],
        'Hungary':['Hungarian','Eastern European'],
        'Ireland':['Irish'],
        'Denmark':['Scandinavian'],
        'Greece':['Greece'],
        'Norway':['Scandinavian'],
        'Finland':['Scandinavian'],
        'Slovakia':['Eastern European'],
        'Luxembourg':['Eastern European'],
        'Slovenia':['Slovenian','Eastern European']
    }
    loc_cus = local_cuisine_by_country[row['country']]
    for l in loc_cus:
        if l in row['Cuisine_'] and l != '':
            return 1
    else:
        return 0
    return 0

def is_pop_cuisine(row):
    pop_cuisine = ['Cafe','Grill','International','Mediterranean','Fast Food','Pizza','Pub','Sushi']
    
    for l in pop_cuisine:
        if l in row['Cuisine_'] and l != '':
            return 1
    else:
        return 0
    return 0

def is_veget_cuisine(row):
    veget_cuisine = ['Vegetarian Friendly', 'Healthy', 'Vegan Options', 'Gluten Free Options']
    
    for l in veget_cuisine:
        if l in row['Cuisine_'] and l != '':
            return 1
    else:
        return 0
    return 0

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    
    #df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    df_output['rest_id']=df_output['Restaurant_id'].apply(lambda x: x[3:])
    df_output['id_ta']=df_output['ID_TA'].apply(lambda x:int(x[1:]))
    rurl_id = re.compile('Review-g(\d+)-')
    df_output['url_']=df_output['URL_TA'].apply(lambda s: rurl_id.findall(s)[0])
    
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['Number of Reviews']).astype('uint8')
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    # тут ваш код по обработке NAN
    # ....
    df_output['nreviews_norm']=(df_output['Number of Reviews']-df_output['Number of Reviews'].min())/(df_output['Number of Reviews'].max()-df_output['Number of Reviews'].min())
    
#CORR
#    df_output.drop(['Number of Reviews'], axis=1, inplace=True)
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    #Перенесена ниже
    #df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    # тут ваш код не Encoding фитчей
    
    # Price Range
    df_output['Price_']=df_output['Price Range'].apply(conv_price)
    price_mean = df_output['Price_'].dropna().mean()
#CORR: Уровень цен 
    price_top = df_output['Price_'].dropna().nlargest(1).iloc[0]
    df_output['Price_'].fillna(price_top, inplace=True)

    global cities, ranks, wordcounts    
    
    # City -> Country, capital, population
    df_output['City'] = df_output['City'].apply(lambda x: 'Porto' if x =='Oporto' else x)
#CORR: Убираем города    
    gen_valcols('City', df_output)

    cities = cities.drop(cities[cities.country == 'United States'].index, axis=0)
    cities = cities.drop(cities[cities.country == 'Canada'].index, axis=0)
    cities = cities.drop(cities[cities.country == 'Venezuela'].index, axis=0)
    df_output=df_output.merge(cities[['city_ascii','capital','country','population', 'lat', 'lng']], how='left', left_on='City', right_on='city_ascii')
    
    df_output['pop_n']=norm(df_output, 'population')
# CORR: Убираем координаты
    df_output['lat_n']=norm(df_output, 'lat')
    df_output['lng_n']=norm(df_output, 'lng')
#    df_output.drop(['population','lat','lng'], axis=1, inplace=True)
    df_output.drop(['lat','lng'], axis=1, inplace=True)
    
    # Cuisine Style
    df_output['Cuisine_']=df_output['Cuisine Style'].apply(conv_cuisine)
    df_output['Cuisine_count'] = df_output['Cuisine_'].apply(lambda x: len(x))
    df_output['Cuisine_count_n']=norm(df_output,'Cuisine_count')
    df_output.drop(['Cuisine_count'], axis=1, inplace=True)
    
    df_output['local_cuisine']=df_output.apply(is_local_cuisine, axis=1)
    df_output['pop_cuisine']=df_output.apply(is_pop_cuisine, axis=1)
    df_output['veget_cuisine']=df_output.apply(is_veget_cuisine, axis=1)
    #gen_valcols('Cuisine_', df_output)
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    # ....
    
#CORR: Страна?
#    df_output=pd.get_dummies(df_output, columns=['capital', 'country'], dummy_na=True)
    df_output=pd.get_dummies(df_output, columns=['country'], dummy_na=True)
    
    
    
    #Rankings adds
    df1 = df_output.merge(ranks, left_on='URL_TA', right_on='url', how='left')
    df_output = df1
    df_output['closed']=df_output.apply(is_closed, axis=1)
    df_output['top_rank']=df_output.apply(top_rank, axis=1)
    df_output.drop(['rank1', 'rank2', 'url', 'rank_main'], axis=1, inplace=True)
    
    ranks_by_city = df_output.groupby('City')[['Ranking', 'top_rank', 'Number of Reviews', 'population']].agg(['max', 'min'])
    df_output['rank_norm'] = df_output[['Ranking', 'City']].apply(get_rank_norm, axis=1, field='Ranking', ranks_by_city=ranks_by_city)

    df_output['rank_top_norm'] = df_output[['top_rank', 'City']].apply(get_rank_norm, axis=1, field='top_rank', ranks_by_city=ranks_by_city)
    
#    df_output['nreviews_norm'] = df_output[['Number of Reviews', 'City']].apply(get_rank_norm, axis=1, field='Number of Reviews', ranks_by_city=ranks_by_city)
    
    df_output.drop(['Ranking', 'top_rank', 'Number of Reviews', 'population'], axis=1, inplace=True)
        
    
    #Reviews
    # - Base
    df_output=df_output.apply(extr_rev, axis=1)
    
    # - Dates
    date_min = min([df_output.date1.min(),df_output.date2.min()])
    date_max = max([df_output.date1.max(),df_output.date2.max()])
    df_output['date_delta_sec']=df_output[['date1','date2']].apply(get_date_delta, axis=1, date_min=date_min, date_max=date_max)
    df_output['last_date_delta']=df_output[['date1','date2']].apply(get_last_date_delta, axis=1, date_min=date_min, date_max=date_max)
    df_output['first_date_delta']=df_output[['date1','date2']].apply(get_first_date_delta, axis=1, date_min=date_min, date_max=date_max)

#CORR Даты    
    df_output['date_delta_n']=norm(df_output,'date_delta_sec')
    df_output['last_date_delta_n']=norm(df_output,'last_date_delta')
    df_output['first_date_delta_n']=norm(df_output,'first_date_delta')

    df_output['date_n_mult']=df_output['date_delta_n'] * df_output['first_date_delta_n']
    df_output.drop(['date_delta_n','last_date_delta_n','first_date_delta_n'],axis=1, inplace=True)
    
    df_output.drop(['date_delta_sec','last_date_delta','first_date_delta'],axis=1, inplace=True)

    # - Words in Review
    df_output=get_dummies_words(df_output, 'rev1',wordcounts.word)
    df_output=get_dummies_words(df_output, 'rev2',wordcounts.word, init=False)
    
    df_output['word_COUNT_n'] = norm(df_output, 'word_COUNT')
    df_output.drop(['word_COUNT'], axis=1, inplace=True)
    
    #df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes in ['object','<m8[ns]','<M8[ns]']]
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
y_pred=list(map(round_rating,y_pred))

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,14)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission=list(map(round_rating,predict_submission))

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
